# Programming Fundamentals & OOP in Python
In this notebook you'll learn:
- Functions: definitions, parameters, return values, scope
- Docstrings & type hints
- Higher-order functions, lambdas, closures, decorators (intro)
- Classes & objects: attributes, methods, special methods
- Class vs. instance attributes; class/static methods; properties
- Dataclasses
- Inheritance, polymorphism, composition
- Equality, ordering, and hashing (dunder methods)
- Practical exercises

## 0) Why Functions and OOP?
- **Functions**: name reusable logic; make code clear and testable.
- **OOP**: bundle data **(state)** and behavior **(methods)** in objects; model real-world entities.


## 1) Functions — Fundamentals


In [1]:
# A simple function
def greet(name):
    return f"Hello, {name}!"

greet("Vanessa")


'Hello, Vanessa!'

In [2]:
# Parameters, default values, keyword arguments
def power(base, exp=2):
    return base ** exp

print(power(5))         # 25
print(power(2, exp=8))  # 256

25
256


In [3]:
# Multiple return values (actually a tuple)
def min_max_avg(nums):
    total = sum(nums)
    return min(nums), max(nums), total / len(nums)

mn, mx, avg = min_max_avg([2, 8, 10, 4])
mn, mx, round(avg, 2)


(2, 10, 6.0)

In [4]:
# Variable-length args: *args and **kwargs
def demo_args(*args, **kwargs):
    return args, kwargs

demo_args(1, 2, a=3, b=4)


((1, 2), {'a': 3, 'b': 4})

### Scope rules
- Names defined inside a function are **local**.
- Names defined at top level are **global** (avoid mutating globals).
- `nonlocal` lets inner functions modify outer (enclosing) scope variables.


In [5]:
x = 10  # global

def use_local():
    x = 99  # shadows global
    return x

def read_global():
    return x

use_local(), read_global(), x


(99, 10, 10)

## 2) Docstrings & Type Hints
- **Docstrings** explain *what* a function/class does, arguments, returns.
- **Type hints** help tools & readers (no runtime enforcement by default).


In [7]:
def normalize(values: list[float], offset: float = 0.0) -> list[float]:
    """
    Normalize a list by subtracting the mean and adding an optional offset.

    Args:
        values: A list of numeric values.
        offset: A value to add after centering (default: 0.0).

    Returns:
        A new list with mean-centered values plus offset.
    """
    if not values:
        return []
    m = sum(values) / len(values)
    return [(v - m) + offset for v in values]

normalize([1.0, 2.0, 3.0], offset=0.5)


[-0.5, 0.5, 1.5]

### De aquí en adelante los temas son un poco avanzados, no es necesario que entiendas 100% la primera vez. Tampoco son cosas que se usen muy a menudo en manejo de datos. Pero debes saber que existen 

## 3) Higher-Order Functions, Lambdas, Closures


In [8]:
# Lambda + map/filter examples
nums = [1, 2, 3, 4, 5, 6]
squared = list(map(lambda n: n * n, nums))
evens = list(filter(lambda n: n % 2 == 0, nums))
squared, evens


([1, 4, 9, 16, 25, 36], [2, 4, 6])

In [9]:
# Closure: a function capturing variables from an enclosing scope
def make_multiplier(k: int):
    def mul(x: int) -> int:
        return k * x
    return mul

double = make_multiplier(2)
triple = make_multiplier(3)
double(10), triple(10)


(20, 30)

## 4) Decorators (Intro)
A **decorator** takes a function and returns a new function with extra behavior.


In [10]:
import time
from functools import wraps

def timing(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        t0 = time.perf_counter()
        try:
            return fn(*args, **kwargs)
        finally:
            dt = (time.perf_counter() - t0) * 1000
            print(f"{fn.__name__} took {dt:.2f} ms")
    return wrapper

@timing
def slow_add(a, b):
    time.sleep(0.05)
    return a + b

slow_add(2, 3)


slow_add took 53.76 ms


5

## 5) Classes & Objects — Basics
A class defines a **blueprint**; an object (instance) holds data and methods.

We'll model a **Track**.


In [11]:
class Track:
    def __init__(self, title: str, artist: str, bpm: int | None = None):
        self.title = title
        self.artist = artist
        self.bpm = bpm  # instance attribute

    def describe(self) -> str:
        bpm_txt = f" @ {self.bpm} bpm" if self.bpm is not None else ""
        return f"{self.title} by {self.artist}{bpm_txt}"

    def set_bpm(self, bpm: int) -> None:
        if bpm <= 0:
            raise ValueError("BPM must be positive")
        self.bpm = bpm

t = Track("Autumn Leaves", "Kosma", bpm=120)
t.describe()


'Autumn Leaves by Kosma @ 120 bpm'

### `__init__`, `__repr__`, `__str__`
- `__init__`: constructor (initialization)
- `__repr__`: unambiguous representation (debugging)
- `__str__`: user-friendly string


In [12]:
class Point:
    def __init__(self, x: float, y: float):
        self.x, self.y = x, y

    def __repr__(self) -> str:
        return f"Point(x={self.x!r}, y={self.y!r})"

    def __str__(self) -> str:
        return f"({self.x}, {self.y})"

p = Point(1.2, 3.4)
repr(p), str(p)


('Point(x=1.2, y=3.4)', '(1.2, 3.4)')

In [13]:
class Playlist:
    default_public = False  # class attribute (shared)

    def __init__(self, name: str, public: bool | None = None):
        self.name = name
        self.public = Playlist.default_public if public is None else public
        self.tracks: list[Track] = []

    def add(self, track: Track) -> None:
        self.tracks.append(track)

    @classmethod
    def make_public_by_default(cls):
        cls.default_public = True

    @staticmethod
    def seconds_to_minsec(seconds: int) -> str:
        return f"{seconds // 60}:{seconds % 60:02d}"

pl = Playlist("Morning Mix")
pl.add(Track("Blue in Green", "Davis", bpm=60))
pl.public, Playlist.default_public, Playlist.seconds_to_minsec(185)


(False, False, '3:05')

## 7) Properties — Computed & Validated Attributes
Use `@property` to expose a computed attribute or attach validation on set.


In [14]:
class Meter:
    def __init__(self, beats: int, unit: int):
        self._beats = beats
        self._unit = unit

    @property
    def beats(self) -> int:
        return self._beats

    @beats.setter
    def beats(self, value: int):
        if value <= 0:
            raise ValueError("beats must be positive")
        self._beats = value

    @property
    def signature(self) -> str:
        return f"{self._beats}/{self._unit}"

m = Meter(4, 4)
m.signature, m.beats


('4/4', 4)

## 8) Dataclasses — Less Boilerplate
`dataclasses` auto-generate `__init__`, `__repr__`, `__eq__`, etc.


In [15]:
from dataclasses import dataclass, field

@dataclass
class Artist:
    name: str
    followers: int = 0
    tags: list[str] = field(default_factory=list)

a1 = Artist("Alice", followers=1200, tags=["jazz", "guitar"])
a1  # nice repr


Artist(name='Alice', followers=1200, tags=['jazz', 'guitar'])

## 9) Inheritance & Polymorphism
- Inheritance: derive a class from a base to reuse/extend behavior.
- Polymorphism: code that works with objects of different classes via a shared interface.


In [16]:
class Instrument:
    def play(self) -> str:
        return "Playing some instrument"

class Guitar(Instrument):
    def play(self) -> str:
        return "Strum 🎸"

class Piano(Instrument):
    def play(self) -> str:
        return "Arpeggio 🎹"

def perform(instr: Instrument):
    print(instr.play())

perform(Guitar())
perform(Piano())


Strum 🎸
Arpeggio 🎹


## A Small OOP Example — Library of Tracks


In [18]:
class Library:
    def __init__(self):
        self._tracks: list[Track] = []

    def add(self, track: Track):
        self._tracks.append(track)

    def by_artist(self, artist: str) -> list[Track]:
        return [t for t in self._tracks if t.artist.lower() == artist.lower()]

    def __len__(self) -> int:
        return len(self._tracks)

lib = Library()
lib.add(Track("So What", "Davis", 140))
lib.add(Track("Freddie Freeloader", "Davis"))
lib.add(Track("Naima", "Coltrane", 66))

len(lib), [t.describe() for t in lib.by_artist("davis")]


(3, ['So What by Davis @ 140 bpm', 'Freddie Freeloader by Davis'])

# Exercises: Functions, Docstrings, Type Hints, and Basic OOP

---

## Part 1: Functions (Fundamentals, Docstrings, Type Hints)

#### 1. Write a function `circle_area(radius: float) -> float`  
- Use type hints and a docstring explaining parameters and return value.  
- The function should compute and return the area of a circle.  
- Formula: π * r² (use `math.pi`).  
- Example: `circle_area(2)` → `12.566...`

---

#### 2. Write a function `average(values: list[float]) -> float`  
- Include a docstring.  
- Handle the case when the list is empty (return `0.0`).  
- Example: `average([10, 20, 30])` → `20.0`  

---

#### 3. Write a function `safe_divide(a: float, b: float) -> float | None`  
- Use type hints and a docstring.  
- Return `None` if `b == 0`, otherwise return the division.  
- Example: `safe_divide(10, 2)` → `5.0`  
- Example: `safe_divide(10, 0)` → `None`  

---

#### 4. Write a function `describe_numbers(nums: list[int]) -> dict[str, float]`  
- Include a docstring.  
- Return a dictionary with:  
  - `"count"` → number of items  
  - `"sum"` → sum of numbers  
  - `"avg"` → average (0 if list empty)  
- Example: `describe_numbers([1,2,3])` → `{"count": 3, "sum": 6, "avg": 2.0}`  

---

#### 5. Write a function `clip(value: float, low: float, high: float) -> float`  
- Include type hints and a docstring.  
- Return `low` if `value < low`, `high` if `value > high`, else return `value`.  
- Example: `clip(15, 0, 10)` → `10`  

---

## Part 2: Basic OOP (Classes, Objects, Attributes, Methods)

#### 6. Create a class `Car`  
- Attributes: `brand` (string), `speed` (int, default 0).  
- Method: `accelerate(amount: int)` that increases speed.  
- Method: `brake(amount: int)` that decreases speed but never below 0.  
- Example:  
```python
c = Car("Toyota")
c.accelerate(50)
c.speed   # 50
c.brake(20)
c.speed   # 30
```

#### 7. Create a class `Student`  

- Attributes: `name` (string), `grades` (list of floats, default empty).  
- Method: `add_grade(grade: float)` to add a grade.  
- Method: `average()` to return average grade (0 if no grades).  

Example:  
```python
s = Student("Ana")
s.add_grade(4.5)
s.add_grade(3.5)
s.average()   # 4.0
```

#### 8. Create a class `Rectangle`

- Attributes: `width` and `height`.  
- Method: `area()` that returns `width × height`.  
- Method: `perimeter()` that returns `2 × (width + height)`.  

Example:  
```python
r = Rectangle(4, 6)
r.area()       # 24
r.perimeter()  # 20
```